# Preamble

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport
# %reload_ext autoreload

import sys 
from os.path import abspath
path = abspath('..')

if path not in sys.path: 
    sys.path.insert(0, path)

In [ ]:
from src.frames import plots
from src.utils import const
from src.frames import catalogs
from src.frames.params import Param
from src.visualization import plot_funcs
from src.visualization import plots1

import numpy as np
import matplotlib.pyplot as plt 

plt.rc("text", usetex=True)

# Load catalog, Check size

In [ ]:
out_file = const.data_path.joinpath('processed/bolshoi_base1.csv')

In [ ]:
# cat_file = const.data_path.joinpath('raw/bolshoi/hlist_1.00035.csv')
# hcat = catalogs.HaloCatalog(cat_file, "Bolshoi", verbose=True, subhalos=False)

# # load catalog 
# hcat.load_base_cat()

# # save base cat for future use. 
# hcat.save_cat(out_file)

In [ ]:
#reload 
hcat = catalogs.HaloCatalog.create_from_saved_cat(out_file, "Bolshoi", verbose=True, subhalos=False)

print(hcat.get_cat().colnames)
print("number of MB being used as estimated in numpy: ", np.array(hcat.get_cat()).nbytes/1e6)

In [ ]:
#sanity check masses look ok. 
plt.hist(np.log10(hcat.get_cat()['mvir']))

# Tests

## frames 

In [ ]:
from src.frames import filters

In [ ]:
#check that everything works as expected. 
print(len(hcat))
hcat.with_filters(filters.catalog_mass_filter(hcat.catalog_name))
print(len(hcat))
hcat.reset_base_cat()
print(len(hcat))

## sanity check relaxedness

In [ ]:
hcat.with_relaxed_filters("power2011")
print(len(hcat))
plt.hist(hcat.get_cat()['xoff'], label='relaxed', histtype='step', density=True)

hcat.reset_base_cat()
print(len(hcat))

plt.hist(hcat.get_cat()['xoff'], label='all', histtype='step', density=True)
plt.legend()

## Plotting demo

In [ ]:
plt.rc("text", usetex=True)
bparams = [(Param('mvir', log=True), Param('T/|U|', log=True))]
bplot = plots.BiPlot(plot_funcs.scatter_binning, bparams, nrows=1, ncols=1)
bplot.run(hcat.cat, nxbins=10, xlabel_size=24, ylabel_size=24, tick_size=18, no_bars=True, show_bands=True,
              legend_label='all galaxies')
bplot.save("test1.png")

In [ ]:
plt.hist(np.log10(hcat.cat['mvir']))
plt.yscale('log')

In [ ]:
len(hcat.cat)

In [ ]:
plots1.test_plot(hcat=hcat)

## Plot test 1 demo

In [ ]:
plt.rc("text", usetex=True)

In [ ]:
hist_kwargs1 = dict(bins=30, histtype='step')
binning_kwargs1 = dict(nxbins=10, no_bars=True, show_bands=True)
general_kwargs1 = dict(xlabel_size=28, ylabel_size=28)

In [ ]:
# (1) histogram
params = [Param('mvir', log=True)]
uplot1 = plots.UniPlot(plot_funcs.histogram, params, nrows=1, ncols=1, tick_size=22)
hcat.reset_base_cat()

uplot1.generate(hcat.get_cat(), legend_label=hcat.catalog_label, color='r', **hist_kwargs1, **general_kwargs1)

hcat.with_relaxed_filters("power2011")
uplot1.generate(hcat.get_cat(), legend_label=hcat.catalog_label, color='b', **hist_kwargs1, **general_kwargs1)


## Test caching

In [ ]:
params = [Param('mvir', log=True)]
colors = ['r','b', 'g']
general_kwargs = dict(xlabel_size=28, ylabel_size=28)
hist_kwargs = dict(bins=30, histtype='step', extra_hist_kwargs=dict(), log_y=True, **general_kwargs)
plot1 = plots.Histogram(plot_funcs.histogram, params, nrows=1, ncols=1, plot_kwargs=hist_kwargs)
hcats = [hcat, 
         catalogs.HaloCatalog.create_relaxed_from_base(hcat, 'power2011'), 
         catalogs.HaloCatalog.create_relaxed_from_base(hcat, 'neto2007')]

for _hcat, color in zip(hcats, colors) : 
    plot1.load_arguments(_hcat.get_cat(), legend_label=_hcat.catalog_label, color=color)

plot1.generate_from_cached()


# Make all plots 

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
hcats = [hcat, catalogs.HaloCatalog.create_relaxed_from_base(hcat, 'power2011'), 
         catalogs.HaloCatalog.create_relaxed_from_base(hcat, 'neto2007')]
colors = ['r','b', 'g']
print([len(hcat1) for hcat1 in hcats])

In [ ]:
out='test1.pdf'
plt.rc("text", usetex=True)
with PdfPages(const.figure_path.joinpath(out)) as pdf:
    plots1.plot_multiple_basic(hcats, pdf, colors)
    plots1.plot_correlation_matrix_basic(hcats, pdf)
    plots1.plot_decades_basic(hcats, pdf, colors=colors)
